# Sistemas Inteligentes

## Procura não informada

## Conteúdos

##### Procura em árvores
* Procura em largura primeiro
* Procura em profundidade primeiro
* Estados vs nós da árvore de procura (classe Node)
* Algoritmo genérico para a largura-primeiro e para a profundidade-primeiro
* Ciclos na procura
* Procura com aprofundamento progressivo
* Exercícios




Nesta aula vamos ver mais alguns aspetos relacionados com a resolução de problemas de acordo com o Paradigma do Espaço de Estados, usando a implementação disponibilizada pelo módulo searchPlus.py, introduzido nas aulas anteriores. Vamos hoje ver como poderemos utilizar os diferentes algoritmos de procura, os "cegos" ou "não informados", alguns deles garantem soluções de menor custo, outros nem por isso.

## Introdução
Lembremos que para resolver um problema segundo o paradigma do espaço de estados, teremos que:


1.   Definir o problema em termos de estados e operadores (com os custos), definir o estado inicial e a condição para ser final.
2.   Utilizar um algoritmo de procura para o resolver.

Vamos experimentar os vários algoritmos de procura, começando por usar um espaço de estados correspondente a um grafo abstracto.


## Recursos necessários

Para executar as experiências que se seguem, 

*   Copie o módulo searchPlus.py para a directoria de trabalho;
*   Copie para o mesmo local os outros módulos auxiliares necessários: utils.py.

## Grafo com custos heterogéneos
Considere o grafo na figura em baixo, com custos heterogéneos. Optimizar não significa uma solução no menor número de acções, porque há diferentes custos para as acções, optimizar é encontrar uma das soluções de menor custo acumulado, entre o estado inicial I e o final F.

![figura](figura-ee-T-2.png)

### Representação dos estados
Neste caso uma letra serve para representar os estados. Num dicionário teremos explicitamente representados todos os estados e operadores do espaço de estados.
Vamos para a definição da classe

In [1]:
from searchPlus import *

class ProblemaGrafoCustos(Problem):
    def __init__(self,initial = 'I', final = 'F',grafo = {'I':{'A':2,'B':5},
             'A':{'C':2,'D':4,'I':2},
             'B':{'D':1,'F':5,'I':5},
             'C':{},
             'D':{'C':3,'F':2},
             'F':{}}) :
        super().__init__(initial,final)
        self.grafo = grafo
        
    def actions(self,estado) :
        sucessores = self.grafo[estado].keys()  # métodos keys() devolve a lista das chaves do dicionário
        accoes = list(map(lambda x : "ir de {} para {}".format(estado,x),sucessores))
        return accoes

    def result(self, estado, accao) :
        """Assume-se que uma acção é da forma 'ir de X para Y'
        """
        return accao.split()[-1]
    
    def path_cost(self, c, state1, action, state2):
        return c + self.grafo[state1][state2]

Comecemos por criar uma instância da classe do problema e mostrar o grafo e o estado inicial.

In [27]:
p = ProblemaGrafoCustos()
print(p.initial)
print(p.grafo)

I
{'I': {'A': 2, 'B': 5}, 'A': {'C': 2, 'D': 4, 'I': 2}, 'B': {'D': 1, 'F': 5, 'I': 5}, 'C': {}, 'D': {'C': 3, 'F': 2}, 'F': {}}


### Largura-Primeiro (árvore)
Vamos executar uma procura em largura primeiro que **dá preferência aos nós da fronteira que estejam mais à superfície (menor número de acções)** e que nos dará uma solução óptima em termos do número de acções, e não em termos do custo. **Só garante a optimalidade quando os custos são homogéneos, o que não é o caso.
A procura em largura, numa árvore, não entra em ciclo** porque precisamente varre os estados em largura: primeiro explora todos os estados à distância de uma acção do estado inicial, depois todos os estados à distância de duas acções do estado inicial, etc. etc., escapando a um ciclo infinito desde que haja uma solução.

In [28]:
resultado = breadth_first_tree_search(p)

Os algoritmos de procura devolvem sempre um objecto da classe **Node** que satisfaz o **goal_test**. É a partir desse ***Node*** que se obtém a solução: a lista de acções que nos leva do estado inicial ao final.
Para obtermos a solução, teremos de aplicar o método ***solution()***. Mais tarde olharemos com mais cuidado para essa classe e para os seus atributos e métodos.

In [29]:
print(resultado.solution())

['ir de I para B', 'ir de B para F']


## Casos em que não há solução

Neste grafo, não há solução a partir do estado C.

Vejamos o que acontece se invocarmos um dos algoritmos de procura a partir de C:

In [30]:
p = ProblemaGrafoCustos('C')
resultado = breadth_first_tree_search(p)
print(resultado.solution())

AttributeError: 'NoneType' object has no attribute 'solution'

Quando não há solução o resultado é do tipo ***None***.
Convém invocar o métdo ***solution()*** apenas quando se encontrar uma solução:

In [38]:
if resultado:
    print("Solução, com custo ",resultado.path_cost)
    for x in resultado.solution():
        print("  " + x)
else:
    print("Sem solução!!!!")

Sem solução!!!!


### Profundidade (árvore)
O algoritmo de profundidade primeiro, **dá preferência aos nós da fronteira que estejam mais afastados da raíz (maior número de acções).** Se executarmos a procura em profundidade, **entraremos em ciclo, porque este método de procura não faz controlo de ciclos.** O primeiro sucessor de I é A e o primeiro sucessor de A é I. (A ordem dos sucessores é invertida na fronteira como verão mais abaixo).

Notem que a procura em profundidade quando devolve uma solução não garante que ela seja a de menor custo, não sendo optimal. **Só garante a solução óptima se todas as soluções estiverem à mesma profundidade e os custos são homogéneos.**

Recomecemos de novo criando uma nova instância do problema e apliquemos a profundidade. Descomentem o código e corram, mas cuidado que vai entrar em ciclo.

Depois de confirmarem que está em ciclo, seleccionem **Kernel** e **Interrupt**.

In [ ]:
# p = ProblemaGrafoCustos()
# resultado = depth_first_tree_search(p)
# print(resultado.solution())

Vamos eliminar os ciclos do grafo (tornando os arcos bidireccionais {I <-> A e I <-> B} em unidireccionais: {I -> A e I -> B}),vamos criar um novo problema e invocar de novo a procura em profundidade-primeiro em árvore, que, por acaso, dá a mesma solução que a largura.

In [39]:
safe={'I':{'A':2,'B':5},'A':{'C':2,'D':4},'B':{'D':1,'F':5},'C':{},'D':{'C':3,'F':2},'F':{}}

In [76]:
ui=ProblemaGrafoCustos(grafo=safe)
resultado = depth_first_tree_search(ui)
print(resultado.solution())


['ir de I para B', 'ir de B para F']
[<Node I>, <Node B>, <Node F>]


Mais abaixo iremos perceber porque é que o método ***depth_first_tree_search()*** não respeita a ordem das acções

### A árvore de procura e a classe Node
A procura desenvolve-se numa árvore de procura formada por nós (objectos da classe ***Node***). Uma coisa são os estados do espaço de estados, outra coisa, completamente diferente, são os nós da árvore de procura. **A cada nó corresponde um estado, mas nós diferentes podem corresponder ao mesmo estado,** porque pode haver vários caminhos entre o estado inicial e um determinado estado.

#### Atributos
Vejemos os atributos da classe Node e o seu construtor:

    parent:o objecto node pai (o objecto node de quem é sucessor);
    state: o estado;
    Notem que se chegámos ao mesmo estado através de dois ramos diferentes, então existem dois Nodes com o mesmo estado.
    action: a acção que nos trouxe até este estado
    path_cost: o custo total do ramo (caminho desde a origem até este estado).
    
Vejemos o construtor

```python
def __init__(self, state, parent=None, action=None, path_cost=0):
        """Create a search tree Node, derived from a parent by an action."""
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1
```

Qualquer árvore de procura vai iniciar-se com a fronteira constituída apenas pela sua raíz, um objecto da classe Node com o estado inicial, sem **parent**, sem **action** e com o **path_cost** a 0. Esse nó só recebe um parâmetro, que é o estado e os restantes atributos ficarão com os valores por omissão.

Vamos construir um **Node** correspondente à raíz da árvore de procura, com o estado inicial:

In [17]:
raiz=Node(ui.initial)
print(raiz)

<Node I>


Vamos mostrar o conteúdo desse nó:

In [18]:
print("Estado:",raiz.state)
print("Acção:",raiz.action)
print("Nó pai:",raiz.parent)
print("Custo do caminho:",raiz.path_cost)

Estado: I
Acção: None
Nó pai: None
Custo do caminho: 0


Notem mais uma vez que qualquer objecto da classe **Node** corresponde a um ramo da árvore de procura. O seu atributo **state** corresponde ao estado folha desse caminho, mais longe da raíz. Para ir trepando pelo ramo, (de ***parent*** em ***parent***) e obtermos todos os estados do ramo ou a sequência de acções, até à raíz, existem os métodos ***path()*** e ***solution()***.

Se os aplicarmos ao nó raíz, obtemos, um caminho só com o nó raíz e uma lista vazia de acções:

In [19]:
print(raiz.path())
print(raiz.solution())

[<Node I>]
[]


Mas se os aplicarmos ao nó devolvido pelos algoritmos de procura, em que o estado corresponde a um estado final, obteremos a solução do problema.

In [20]:
print("Solução:",resultado.solution())
print("Ramo:",resultado.path())

Solução: ['ir de I para B', 'ir de B para F']
Ramo: [<Node I>, <Node B>, <Node F>]


Vamos inspeccionar os métodos ***solution()*** e ***path()***.

``` python
    def solution(self):
        """Return the sequence of actions to go from the root to this node."""
        return [node.action for node in self.path()[1:]]

    def path(self):
        """Return a list of nodes forming the path from the root to this node."""
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))
```
Notem que o método ***path()*** vai recolhendo numa lista os nós desde a folha, subindo pelo atributo **parent**, e depois essa lista é invertida.

O método ***solution()*** devolve numa lista a acção para cada nó que faz parte da lista devolvida por ***path()***, ignorando a primeira acção que é **None**.

Continuemos. 
Vamos devolver uma solução mais **pretty**, que nos devolve também o custo, fazendo uso do atributo **path_cost** da classe **Node**.

In [21]:
print("Solução, com custo",resultado.path_cost)
for x in resultado.solution():
    print("  " + x)

Solução, com custo 10
  ir de I para B
  ir de B para F


#### Exercício 1
Desenvolvam o código para devolver não só a sequência das acções, mas indiquem o estado inicial e final e também os custos de cada acção e o custo final.
```
Solução, com custo 10
Começamos em: I
  ir de I para B (5)
  ir de B para F (5)
F satisfaz o objectivo
```

In [97]:
# Resolução do exercício 1
print("Solução, com custo", resultado.path_cost)

path = resultado.path()
path_iter = iter(path)
print(f'Começamos em {next(path_iter).state}')

for node in path_iter:
    print(f'\t{node.action} ({node.path_cost - node.parent.path_cost})')

print(f'{path[-1].state} satisfaz o objetivo')

Solução, com custo 10
Começamos em I
	ir de I para B (5)
	ir de B para F (5)
F satisfaz o objetivo


Regressemos à árvore formada apenas pela raíz, o nó com o estado inicial do problema do grafo, I.

In [73]:
raiz

<Node I>

Se quisermos expandir um nó, que vai criar uma lista de novos nós, em que os estados desses nós serão os sucessores do nó referido, pela ordem das acções, usaremos o método ***expand()*** da classe **Node**, que recebe como argumento a instância do problema. Notem que esse método é que fará uso dos métodos implicados na formulação do problema em concreto: ***actions()***, ***result()*** e ***path_cost()***.
Vamos então expandir o nó que acabámos de criar.

In [74]:
novos = raiz.expand(p)
print(novos)

[<Node A>, <Node B>]


Vamos inspeccionar os novos nós.

In [75]:
prim=novos[0]
print("Primeiro sucessor:")
print("estado =",prim.state)
print("pai=",prim.parent)
print("accao =",prim.action)
print("custo do caminho =",prim.path_cost)
seg=novos[1]
print("\nSegundo sucessor:")
print("estado =",seg.state)
print("pai=",seg.parent)
print("accao =",seg.action)
print("custo do caminho =",seg.path_cost)

Primeiro sucessor:
estado = A
pai= <Node I>
accao = ir de I para A
custo do caminho = 2

Segundo sucessor:
estado = B
pai= <Node I>
accao = ir de I para B
custo do caminho = 5


Reparem que os nós sucessores ficaram ligados ao nó pai **Node I**, os custos foram actualizados e a última acção de cada um deles foi guardada.

Vejam o código do método ***expand()*** que faz uso dos métodos da classe **Problem()** ou neste caso da classe **ProblemaGrafoCustos**: ***actions()***, ***result()*** e ***path_cost()***, que aplica ao seu estado a lista de acções, gerando para cada acção, o novo nó sucessor aplicando o método ***child_node()***, e devolve a lista desses nós sucessores.

``` python
    def expand(self, problem):
        """List the nodes reachable in one step from this node."""
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]
```

Vejam o método ***child_node()*** que gera o estado que resulta da acção e cria o novo **Node**, preenchendo os atributos.

``` python
    def child_node(self, problem, action):
        next = problem.result(self.state, action)
        return Node(next, self, action,
                    problem.path_cost(self.path_cost, self.state,
                                      action, next))
```

#### Exercício 2 
Expandam o nó B, o segundo dos sucessores do nó I, e mostrem os atributos dos seus sucessores, através de um ciclo for, testando também se qualquer deles é final

In [84]:
## resolução do exercício 2
exp_b = novos[1].expand(p)
for i, node in enumerate(exp_b):
    print(f'Sucessor {i}:')
    print(f'estado = {node.state}')
    print(f'pai = {node.parent}')
    print(f'accao = {node.action}')
    print(f'custo do caminho = {node.path_cost}\n')

Sucessor 0:
estado = D
pai = <Node B>
accao = ir de B para D
custo do caminho = 6

Sucessor 1:
estado = F
pai = <Node B>
accao = ir de B para F
custo do caminho = 10

Sucessor 2:
estado = I
pai = <Node B>
accao = ir de B para I
custo do caminho = 10



### Algoritmo Genérico para a largura-primeiro e para a profundidade-primeiro
Os algoritmos de procura em profundidade e largura, que recebem como argumento a instância do problema, invocam ambos o método ***tree_search()***, passando-lhe o problema e a fronteira da árvore de procura, inicialmente vazia. No caso da **profundidade usa-se uma pilha, no caso da largura usa-se uma fila para a fronteira**, passadas ambas vazias.

```python
def breadth_first_tree_search(problem):
    """Search the shallowest nodes in the search tree first."""
    return tree_search(problem, FIFOQueue())


def depth_first_tree_search(problem):
    """Search the deepest nodes in the search tree first."""
    return tree_search(problem, Stack())
```

Olhemos com algum detalhe para o método ***tree_search()***.

``` python
def tree_search(problem, frontier):
    frontier.append(Node(problem.initial))
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            return node
        frontier.extend(node.expand(problem))
    return None
```

O ***tree_search()*** recebe dois parâmetros: o problema e a fronteira inicial que está vazia.
    
    1. Inicialmente é adicionada à fronteira vazia um objecto da classe **Node** formado a partir do estado inicial obtido do problema.
    2. Enquanto a fronteira não for vazia:
        2a. Retira o primeiro da fronteira (no caso da pilha ou da fila, o primeiro varia)
        2b. se satisfaz o objectivo devolve, senão expande o nó retirado e estende a fronteira com os novos nós (sucessores)
    3. Se não saiu antes é porque não há solução, (a fronteira está vazia) devolve None -  o espaço de estados foi esgotado
    
Expande-se a fronteira com os novos **Nodes** que resultam da aplicação das acções ao estado do **Node** que foi retirado da fronteira. Reparem que tanto a classe **Stack** como **FIFOQueue** possuem os métodos ***append()*** e ***extend()***. 

Vamos brincar com a **Stack**

In [85]:
x = Stack()
print("Tipo=",type(x))
x.append(1)
print(x)
x.append(2)
print(x)
x.pop()
print(x)

Tipo= <class 'list'>
[1]
[1, 2]
[1]


In [86]:
x.extend([10,11,12])
print(x)
x.pop()
print(x)

[1, 10, 11, 12]
[1, 10, 11]


A classe **Stack** corresponde a uma lista e os elementos são retirados do fim da lista com ***pop()***, inseridos no fim com o ***append()*** e concatenados no fim com ***extend()***. O **FIFOQueue** é uma classe que tem um atributo **queue** que é da classe **deque**, no módulo collections que é uma fila "double-ended" na qual os elementos podem ser tanto inseridos como apagados de ambas as pontas da fila. Brinquemos agora com a **FIFOQueue** e **deque**.

In [87]:
f = FIFOQueue()
print(f.queue)
print(type(f))
print('Adicionemos 2:')
f.append(2)
print(f.queue)
print('Concatenemos [5,6,7]')
f.extend([5,6,7])
print(f.queue)
print('Façamos pop:')
z=f.pop()
print(f.queue)
print('Adicionemos 20:')
f.append(20)
print(f.queue)

deque([])
<class 'utils.FIFOQueue'>
Adicionemos 2:
deque([2])
Concatenemos [5,6,7]
deque([2, 5, 6, 7])
Façamos pop:
deque([5, 6, 7])
Adicionemos 20:
deque([5, 6, 7, 20])


### Porque entrou em ciclo a procura em Profundidade-Primeiro?
Para vermos porque é que a procura em profundidade entrou em ciclo, vamos simular o funcionamento do ***depth_first_tree_search()*** a partir do **Node** I, que está na variável raiz.

In [88]:
fronteira = Stack()
fronteira.append(raiz)
print("Fronteira:",fronteira)

Fronteira: [<Node I>]


A ordem dos sucessores corresponde à ordem dos estados sucessores na representação do grafo no dicionário, mas como é uma pilha, o nó a que fazemos pop() é:

In [90]:
expandir = fronteira.pop()
print("Fazemos POP de",expandir)
print("que expandimos e fazemos push para a fronteira")
fronteira.extend(expandir.expand(p))
print("Fronteira:",fronteira)

Fazemos POP de <Node I>
que expandimos e fazemos push para a fronteira
Fronteira: [<Node A>, <Node B>]


Continuemos a simular a profundidade primeiro. Podemos ver que o primeiro da pilha é o último da lista, o Nó B e que B é o estado que resulta da segunda acção. A ordem dos sucessores é invertida quando se empilha.

In [91]:
expandir = fronteira.pop()
print("Fazemos POP de",expandir)
print("que expandimos e fazemos push para a fronteira de",expandir.expand(p))
fronteira.extend(expandir.expand(p))
print("Fronteira:",fronteira)

Fazemos POP de <Node B>
que expandimos e fazemos push para a fronteira de [<Node D>, <Node F>, <Node I>]
Fronteira: [<Node A>, <Node D>, <Node F>, <Node I>]


Agora, vamos fazer pop() e gerar os novos sucessores que irão para a fronteira empilhadinhos.

In [92]:
prim = fronteira.pop()
prim.solution()

['ir de I para B', 'ir de B para I']

Ao empilharmos os sucessores, a sua ordem inverte-se e o ***Node I*** vai ser o primeiro a ser expandido, dando origem ao ciclo... porque há dois I's no ramo, na raíz e na ponta.

#### Exercício 3
Crie uma nova versão do método ***tree_search()***, o ***tree_echo_search()*** que imprime os estados à medida que vão sendo expandidos. Adapte os dois algoritmos (prof. e larg. prim) de modo a usarem essa nova versão. Teste com o grafo abstracto e tanto com a largura-primeiro como com a procura em profundidade-primeiro (neste caso o grafo sem ciclos).

In [96]:
# Resolução do exercício 3
def imprime_estado(node):
    print(f'estado = {node.state}')
    print(f'pai = {node.parent}')
    print(f'accao = {node.action}')
    print(f'custo do caminho = {node.path_cost}\n')

def tree_echo_search(problem, frontier):
    frontier.append(Node(problem.initial))

    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            return node

        expandidos = node.expand(problem)
        for no in expandidos:
            imprime_estado(no)

        frontier.extend(expandidos)

    return None

def breadth_first_tree_echo_search(problem):
    """Search the shallowest nodes in the search tree first."""
    return tree_echo_search(problem, FIFOQueue())

ui = ProblemaGrafoCustos(grafo=safe)
print('---------- BFS ECHO ---------------')
resultado = breadth_first_tree_echo_search(ui)
print(resultado.solution())


def depth_first_tree_echo_search(problem):
    """Search the deepest nodes in the search tree first."""
    return tree_echo_search(problem, Stack())

print('---------- DFS ECHO ---------------')
resultado = depth_first_tree_echo_search(ui)
print(resultado.solution())

---------- BFS ECHO ---------------
estado = A
pai = <Node I>
accao = ir de I para A
custo do caminho = 2

estado = B
pai = <Node I>
accao = ir de I para B
custo do caminho = 5

estado = C
pai = <Node A>
accao = ir de A para C
custo do caminho = 4

estado = D
pai = <Node A>
accao = ir de A para D
custo do caminho = 6

estado = D
pai = <Node B>
accao = ir de B para D
custo do caminho = 6

estado = F
pai = <Node B>
accao = ir de B para F
custo do caminho = 10

estado = C
pai = <Node D>
accao = ir de D para C
custo do caminho = 9

estado = F
pai = <Node D>
accao = ir de D para F
custo do caminho = 8

estado = C
pai = <Node D>
accao = ir de D para C
custo do caminho = 9

estado = F
pai = <Node D>
accao = ir de D para F
custo do caminho = 8

['ir de I para B', 'ir de B para F']
---------- DFS ECHO ---------------
estado = A
pai = <Node I>
accao = ir de I para A
custo do caminho = 2

estado = B
pai = <Node I>
accao = ir de I para B
custo do caminho = 5

estado = D
pai = <Node B>
accao = ir d

#### Exercício 4
Do modo que está feito o ***tree_search()***, ao usarmos uma Stack, a ordem das acções e da adição à fronteira dos novos estados, não vai ser respeitada, sendo invertida. Altere o ***tree_search()*** para o ***tree_ord_search()*** de modo a que ordem de visita dos estados corresponda à ordem dos sucessores no dicionário, e mostrando a ordem de expansão dos nós apenas para confirmar que a ordem da expansão dos nós corresponde à ordem das acções na procura em profundidade-primeiro!

In [ ]:
# Resolução do exercício 4


#### Exercício 5
A pesquisa em profundidade é muito sensível à ordem das acções. É habitual baralhar a lista de acções de modo aleatório ao expandir os nós. 
Crie uma nova versão do método ***tree_search()***, o ***tree_shuffle_search()*** que resolva esse problema e crie também o ***depth_first_shuffle_tree_search()***.
Teste com o grafo abstracto.

In [ ]:
# Resolução do exercício 5


##### Exercício 6
Construa uma função que invoca a variante da profundidade-primeiro feita no exercício 5, para um problema, um número de vezes, passados ambos por parâmetros e que devolve a média dos tamanhos das diversas soluções encontradas, bem como os comprimentos da menor e da maior solução. Teste esse método com o problema do troco, por exemplo, que está formulado a seguir e em que não há risco de ciclos, calculando para 1000 resoluções por exemplo, confirmando que a profundidade-primeiro é muito sensível à ordem das acções. Compare com a largura-primeiro que garante o troco no menor número de moedas, dando a solução mais curta.

In [98]:
class Troco(Problem):
    """ Vamos ter como estado com um inteiro.
    """
      
    def __init__(self, initial=0, goal=87, moedas = [50,20,10,5,2,1] ):
        self.initial, self.goal, self.moedas = initial, goal, moedas
            
    def actions(self, state):
        """Indicação das peças que vão deslizar..."""
        return [m for m in self.moedas if state+m <= self.goal]
    
    def result(self, state, action):
        """adicionar mais uma moeda  """
        return state + action

In [102]:
# Resolução do exercício 6
ptroco = Troco()
bfs_troco = breadth_first_tree_search(ptroco)
print(bfs_troco.solution())

dfs_troco = depth_first_tree_search(ptroco)
print(dfs_troco.solution())

[50, 20, 10, 5, 2]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


#### Exercício 7
Crie uma versão dos métodos de procura em profundidade-primeiro e largura-primeiro que calculam e imprimam o maior tamanho da fronteira durante o processo de procura.
Compare o consumo de memória para a largura-primeiro e a profundidade-primeiro para o problema do puzzle da formação do troco.

In [ ]:
# Resolução do exercício 7



#### Exercício 8
Inverta a ordem das moedas (acções ou construtor) na formulação do problema do troco e compare os resultados da profundidade-primeiro com os do exercício anterior. Vamos também confirmar que a ordem das acções pode afectar a largura.

In [ ]:
# Resolução do exercício 8


#### Exercício 9
Compare os algoritmos de procura em profundidade-primeiro e de largura-primeiro em termos de tempo tanto para o caso do exercício 7 como 8.

In [ ]:
# resolução do exercício 9  


### Aprofundamento Progressivo (árvore)
Se aplicarmos o aprofundamento progressivo não teremos também a garantia de encontrar a solução óptima porque os custos não são homogéneos. **Este algoritmo garante a solução mais próxima do estado inicial, em termos de número de acções,** como a procura em largura, mas com um preço menor do que a largura em termos de memória utilizada.
Começa por fazer uma procura limitada ao estado inicial e à profundidade 0, depois limitada à profundidade 1, a seguir limitada à profundidade 2, etc. etc., evitando os ciclos até encontrar uma solução que esteja mais próxima da raíz da árvore em termos do número de movimentos ou acções.
Vamos aplicar o aprofundamento progressivo ao grafo abstracto.

In [ ]:
p = ProblemaGrafoCustos()
resultado = iterative_deepening_search(p)
print("solução aprofundamento progressivo (árvore) com custo", str(resultado.path_cost)+":")
for x in resultado.solution():
    print(x)

Vamos aplicar também ao problema dos trocos e calculemos o tempo que demora.

In [ ]:
import timeit
t = Troco()
start = timeit.default_timer()
resultado = iterative_deepening_search(t)
print("solução aprofundamento progressivo (árvore) com custo", str(resultado.path_cost)+":")
print(resultado.solution())
stop = timeit.default_timer()

print('Time: ', stop - start) 

Vamos comparar em termos de tempo com a procura em largura.

In [ ]:
start = timeit.default_timer()

#Your statements here
print("Resultados da Largura:")
res= breadth_first_tree_search(t)
print(res.solution())
print("Solução óptima:",len(res.solution()))

stop = timeit.default_timer()

print('Time: ', stop - start)  

#### Exercício 10
O aprofundamento progressivo do pacote aima-python é o seguinte:

```Python
def iterative_deepening_search(problem):
    """[Figure 3.18]"""
    for depth in range(sys.maxsize):
        result = depth_limited_search(problem, depth)
        if result != 'cutoff':
            return result
```

e começa sempre com a profundidade 1 e progride em incrementos de 1.
Crie uma variante em que a profundidade inicial e o incremento sejam passados por parâmetro.
Teste-a com algum dos problemas.

In [ ]:
# Resolução do exercício 10


#### Exercício 11
Faça uma variante do aprofundamento progressivo de modo a mostrar a sequência de nós expandidos.
Aplique-o ao grafo abstracto com ciclos.

In [ ]:
# Resolução do exercício 11
